In [ ]:
import os

import numpy as np
import pandas as pd

evaluation_dir = os.path.join(os.getcwd(), '..', 'evaluation', 'signals')
predictions_file = os.path.join(evaluation_dir, 'predictions.csv')

predictions = pd.read_csv(predictions_file)
predictions['signal'] = predictions['signal'].apply(eval).apply(np.array)
predictions['parameters'] = predictions['parameters'].apply(eval).apply(dict)
predictions['execution_time'] = pd.to_timedelta(predictions['execution_time'])

ground_truth_file = os.path.join(evaluation_dir, 'ground_truth.csv')
ground_truth = pd.read_csv(ground_truth_file)
ground_truth['signal'] = ground_truth['signal'].apply(eval).apply(np.array)

predictions.head()

In [ ]:
import respiratory_extraction.dataset as repository

dataset = repository.from_default()

In [ ]:
from respiratory_extraction.models.frequency_extraction import FrequencyExtractor

evaluation_results = []

for index, row in predictions.iterrows():
    subject, scenario = row['subject'], row['scenario']

    # Get the ground truth signal and sampling rate
    gt_signal, gt_sampling_rate = ground_truth[
        (ground_truth['subject'] == subject) &
        (ground_truth['scenario'] == scenario)
        ][['signal', 'sampling_rate']].values[0]

    gt_extractor = FrequencyExtractor(
        gt_signal,
        gt_sampling_rate,
        lowpass=0.1,
        highpass=0.6,
    )
    gt_frequencies = {
        'fft': gt_extractor.frequency_from_fft(),
        'pc': gt_extractor.frequency_from_peaks(),
        'cp': gt_extractor.frequency_from_crossing_point(),
        'nfcp': gt_extractor.frequency_from_nfcp(),
    }

    frequency_extractor = FrequencyExtractor(
        row['signal'],
        row['sampling_rate'],
        lowpass=0.1,
        highpass=0.6,
    )
    predicted_frequencies = {
        'fft': frequency_extractor.frequency_from_fft(),
        'pc': frequency_extractor.frequency_from_peaks(),
        'cp': frequency_extractor.frequency_from_crossing_point(),
        'nfcp': frequency_extractor.frequency_from_nfcp(),
    }

    for method in gt_frequencies.keys():
        # The frequency is in Hz, we want to convert it to bpm
        predicted_hz = predicted_frequencies[method] * 60
        gt_hz = gt_frequencies[method] * 60

        evaluation_results.append({
            'subject': subject,
            'scenario': scenario,
            'strategy': row['method'],
            'parameters': row['parameters'],
            'execution_time': row['execution_time'],
            'method': method,
            'gt_frequency': predicted_hz,
            'predicted_frequency': gt_hz,
            'error': abs(gt_hz - predicted_hz),
        })

In [ ]:
evaluation_results = pd.DataFrame(evaluation_results)

In [ ]:
evaluation_results['roi_selection'] = evaluation_results['parameters'].apply(lambda param: param['roi'])

In [ ]:
evaluation_results

In [ ]:
# Plot the mean error for each method
import matplotlib.pyplot as plt

methods = evaluation_results['method'].unique()
mae = []
mse = []

for method in methods:
    method_results = evaluation_results[
        (evaluation_results['method'] == method) &
        (evaluation_results['strategy'] == 'pixel_intensity')]

    mean_absolut_error = method_results['error'].mean()
    mae.append(mean_absolut_error)

    mean_squared_error = (method_results['error'] ** 2).mean()
    mse.append(mean_squared_error)

fig, ax = plt.subplots(1, 2, figsize=(15, 5))

ax[0].bar(methods, mae)
ax[0].set_title('Overall MAE')
ax[0].set_ylabel('Error (bpm)')
ax[0].set_xlabel('Method')

ax[1].bar(methods, mse)
ax[1].set_title('Overall MSE')
ax[1].set_ylabel('Error (bpm)')
ax[1].set_xlabel('Method')